In [16]:
import os
import pandas as pd
from pathlib import Path
from get_all_trades import process_ca, process_data, updatePL
from get_all_trades import calculate_PL
#from get_all_trades import convert_to_df


In [5]:
cwd =  Path.cwd()

folder = cwd /'IBActivityNew'

tables = cwd / 'Tables'

In [6]:
if tables/'tables_Trades.csv':
    print(True)
else:
    print(False)

True


In [7]:
def get_all_trades(folder):
    #Creates a list of all files in folder
    filelist = os.listdir(folder)
    #Creates a empty list to append all dataframes
    dataframe_list = []
    #Iterates through all files in filelist
    asset_categories_lst = ['Stocks','Equity and Index Options','Futures','Options On Futures', 'Forex']
    for a_file in filelist:
        #Checks if file is a .csv
        if a_file.lower().endswith('.csv'):
            data = convert_to_df(a_file)
            for i in asset_categories_lst:
                df = process_data(data, i)
                dataframe_list.append(df)
            dataframe_list.append(process_ca(data))
    new_trades = pd.concat(dataframe_list,sort=False )
    dataframe = pd.concat(dataframe_list,sort=False )
    dataframe['Proceeds'] = dataframe['Proceeds'].astype('float')
    dataframe['Quantity'] = dataframe['Quantity'].astype('float')
    dataframe['T. Price'] = dataframe['T. Price'].astype('float')
    dataframe['Comm/Fee'] = dataframe['Comm/Fee'].astype('float')
    dataframe['Date/Time'] = pd.to_datetime(dataframe['Date/Time'])
    dataframe['Quantity_Rsum'] =  dataframe.groupby(['Symbol'])['Quantity'].transform(lambda x : x.cumsum())
    return dataframe

In [8]:
def convert_to_df(a_file):
    data = pd.read_csv(folder/a_file, names=list('abcdefghijklmnopq'))
    #data.drop(data.loc[(data.a == 'Notes/Legal Notes') | (data.a == 'Codes') ].index, inplace= True)
    #data = (data.a != 'Codes') & (data.a != 'Notes/Legal Notes')
    data = data.loc[~data.a.isin(['Statement','Codes', 'Notes/Legal Notes','Account Information','Change in NAV','Mark-to-Market Performance Summary','Realized & Unrealized Performance Summary','Open Positions'])]
    return data

In [71]:
def calculatePL (df):
    newdf = df.loc[:,['Quantity','Quantity_Rsum','T. Price','Proceeds']]
    newdf['prev'] = newdf['Quantity_Rsum'].shift(1 , fill_value=0)
    #Make a new column OIDC
    oidc_options = [
        newdf['prev'] == 0,
        newdf['Quantity_Rsum'] == 0,
        newdf['prev']/newdf['Quantity_Rsum'] < 0,
        abs(newdf['Quantity_Rsum']) > abs(newdf['prev'])
    ]
    oidc_choices = [
        'Open','Close','Reversal', 'Increase'
    ]
    newdf['oidc'] = np.select(oidc_options, oidc_choices, default = 'Decrease')

    newdf['index'] = np.where((newdf['oidc']== 'Open')|(newdf['oidc']=='Reversal'),1,0)

    newdf['Increase'] = np.where((newdf['oidc'] == 'Open') | (newdf['oidc'] == 'Increase') | (newdf['oidc'] =='Reversal'),1,0)
    newdf['ProceedsIncrease'] = newdf['Proceeds']*newdf['Increase']
    newdf['QuantityIncrease'] = newdf['Quantity']*newdf['Increase']
    newdf['cumsum'] = newdf['index'].cumsum()
    newdf['TotProceedsIncrease']= newdf.groupby(['cumsum'])['ProceedsIncrease'].cumsum()
    newdf['TotQuantityIncrease']= newdf.groupby(['cumsum'])['QuantityIncrease'].cumsum()
    newdf['AvgOpenPrice'] = newdf['TotProceedsIncrease'] / newdf['TotQuantityIncrease'] * -1
    newdf['AvgOpenPriceBefore'] = newdf['AvgOpenPrice'].shift(1 , fill_value=0)

    plOptions = [
        newdf['oidc'] == 'Open',
        newdf['oidc'] == 'Increase',
        newdf['oidc'] == 'Decrease',
        newdf['oidc'] == 'Close',
        newdf['oidc'] == 'Reversal'
    ]
    plChoices = [
        # Open case P/L = 0
        0,
        #Increase case 
        0,
        #Decrease case: Average Open Price - T. Price * Quantity (* Multiplier +coommfee)
        (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity'],
        #Close case 
        (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity'],
        #Reverse case
        (newdf['AvgOpenPriceBefore'] - newdf['T. Price']) * newdf['Quantity']
    ]
    newdf['PL'] = np.select(plOptions, plChoices, default = 0)
    newdf['CumPL'] = newdf['PL'].cumsum()
    newdf.drop(['Quantity',	'Quantity_Rsum', 'T. Price', 'Proceeds', 'prev', 'oidc', 'index','Increase','ProceedsIncrease', 'QuantityIncrease', 'cumsum', 'TotProceedsIncrease','TotQuantityIncrease','AvgOpenPriceBefore'], axis=1, inplace=True)
    #return newdf['PL'], newdf['AvgOpenPrice'], newdf['CumPL']
    return newdf

def updatePL (df):
    df.drop(['AvgOpenPrice','PL','CumPL'], axis=1, inplace = True, errors= 'ignore')
    dfgrouped = df.groupby('Symbol')
    dataframe_list = []
    for name, group in dfgrouped:
        symbol_group = dfgrouped.get_group(name)
        profLoss = calculatePL( symbol_group)
        symbol_group = symbol_group.join(profLoss)
        dataframe_list.append(symbol_group)
    new_trades = pd.concat(dataframe_list,sort=False ).sort_index()
    return new_trades


In [12]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 14 columns):
Unnamed: 0        1336 non-null int64
Unnamed: 0.1      324 non-null float64
Asset Category    1336 non-null object
Currency          1336 non-null object
Symbol            1336 non-null object
Date/Time         1336 non-null object
Quantity          1336 non-null float64
T. Price          1336 non-null float64
Proceeds          1336 non-null float64
Comm/Fee          1336 non-null float64
Quantity_Rsum     1336 non-null float64
AvgOpenPrice      1336 non-null float64
PL                1336 non-null float64
CumPL             1336 non-null float64
dtypes: float64(9), int64(1), object(4)
memory usage: 146.2+ KB


In [19]:
newtrades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 220 to 1078
Data columns (total 12 columns):
Asset Category    1012 non-null object
Currency          1012 non-null object
Symbol            1012 non-null object
Date/Time         1012 non-null datetime64[ns]
Quantity          1012 non-null float64
T. Price          1012 non-null float64
Proceeds          1012 non-null float64
Comm/Fee          1012 non-null float64
Quantity_Rsum     1012 non-null float64
AvgOpenPrice      1012 non-null float64
PL                1012 non-null float64
CumPL             1012 non-null float64
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 102.8+ KB


In [23]:
#if tables/'tables_Trades.csv':
trades = pd.read_csv(tables / 'tables_Trades.csv')



In [24]:
newtrades = get_all_trades(folder)
newtrades = updatePL(newtrades)


In [25]:
trades = trades.append(newtrades, sort=False)

In [26]:
trades.to_csv(tables/'tables_Trades.csv')